In [1]:
import os
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, make_scorer, f1_score
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor, XGBClassifier

from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical

# import kagglehub

# path = kagglehub.dataset_download("miadul/brain-tumor-dataset")
# os.rename(path, './datasets')

# make data

In [2]:
df = pd.read_csv('./datasets/brain_tumor_dataset.csv')
df.columns = [c.lower() for c in df.columns]

stage_to_num = {'I': 1, 'II': 2, 'III': 3, 'IV': 4}
df['stage'] = df['stage'].apply(lambda x: stage_to_num.get(x))
df['all_symptoms'] = df[['symptom_1', 'symptom_2', 'symptom_3']].apply(list, axis=1)
df.drop(['symptom_1', 'symptom_2', 'symptom_3'], axis=1, inplace=True)
df.drop('patient_id', axis=1, inplace=True)

all_symptoms = set().union(*df['all_symptoms'])

for s in all_symptoms:
    df[s.lower().replace(' ', '_')] = df['all_symptoms'].apply(lambda x: 1 if s in x else 0)

for binary_feature in ['radiation_treatment', 'surgery_performed', 'chemotherapy', 'family_history', 'follow_up_required']:
    df[binary_feature] = df[binary_feature].apply(lambda x: 1 if x == 'Yes' else 0)
    
df['gender'] = df['gender'].apply(lambda x: 1 if x == 'Male' else 0)
df['mri_result'] = df['mri_result'].apply(lambda x: 1 if x == 'Positive' else 0)
    
df.drop('all_symptoms', axis=1, inplace=True)

df.head()

,age,gender,tumor_type,tumor_size,location,histology,stage,radiation_treatment,surgery_performed,chemotherapy,survival_rate,tumor_growth_rate,family_history,mri_result,follow_up_required,seizures,vision_issues,nausea,headache
0,73,1,Malignant,5.375612,Temporal,Astrocytoma,3,0,0,0,51.312579,0.111876,0,1,1,1,1,0,0
1,26,1,Benign,4.847098,Parietal,Glioblastoma,2,1,1,1,46.373273,2.165736,1,1,1,0,0,1,1
2,31,1,Benign,5.588391,Parietal,Meningioma,1,0,0,0,47.072221,1.884228,0,0,0,1,1,0,1
3,29,1,Malignant,1.436600,Temporal,Medulloblastoma,4,1,0,1,51.853634,1.283342,1,0,0,1,1,0,1
4,54,0,Benign,2.417506,Parietal,Glioblastoma,1,0,0,1,54.708987,2.069477,0,1,1,1,0,0,1


In [3]:
df['tumor_type'].unique()

array(['Malignant', 'Benign'], dtype=object)

In [4]:
df['tumor_type'] = df['tumor_type'].apply(lambda x: 1 if x == 'Malignant' else 0)

In [5]:
dummies = pd.get_dummies(df[['location', 'histology']], drop_first=True)
dummies.columns = [c.lower() for c in dummies.columns]

In [6]:
df.drop(['location', 'histology'], axis=1, inplace=True)
df = df.join(dummies)

df.head()

,age,gender,tumor_type,tumor_size,stage,radiation_treatment,surgery_performed,chemotherapy,survival_rate,tumor_growth_rate,...,seizures,vision_issues,nausea,headache,location_occipital,location_parietal,location_temporal,histology_glioblastoma,histology_medulloblastoma,histology_meningioma
0,73,1,1,5.375612,3,0,0,0,51.312579,0.111876,...,1,1,0,0,0,0,1,0,0,0
1,26,1,0,4.847098,2,1,1,1,46.373273,2.165736,...,0,0,1,1,0,1,0,1,0,0
2,31,1,0,5.588391,1,0,0,0,47.072221,1.884228,...,1,1,0,1,0,1,0,0,0,1
3,29,1,1,1.436600,4,1,0,1,51.853634,1.283342,...,1,1,0,1,0,0,1,0,1,0
4,54,0,0,2.417506,1,0,0,1,54.708987,2.069477,...,1,0,0,1,0,1,0,1,0,0


In [7]:
df.isna().sum()

age                          0
gender                       0
tumor_type                   0
tumor_size                   0
stage                        0
radiation_treatment          0
surgery_performed            0
chemotherapy                 0
survival_rate                0
tumor_growth_rate            0
family_history               0
mri_result                   0
follow_up_required           0
seizures                     0
vision_issues                0
nausea                       0
headache                     0
location_occipital           0
location_parietal            0
location_temporal            0
histology_glioblastoma       0
histology_medulloblastoma    0
histology_meningioma         0
dtype: int64

In [8]:
X = df.drop(columns='survival_rate')

In [9]:
y_num = df['survival_rate']

In [10]:
def bucket_survival_rate(x):
    if x <= .5:
        return 0
    if .5 < x <= .75:
        return 1
    if .75 < x <= 1:
        return 2

y_cat = df['survival_rate'].apply(lambda x: bucket_survival_rate(x/100))

In [11]:
y_cat.value_counts()

1    8386
2    8343
0    3271
Name: survival_rate, dtype: int64

In [12]:
df.describe()

,age,gender,tumor_type,tumor_size,stage,radiation_treatment,surgery_performed,chemotherapy,survival_rate,tumor_growth_rate,...,seizures,vision_issues,nausea,headache,location_occipital,location_parietal,location_temporal,histology_glioblastoma,histology_medulloblastoma,histology_meningioma
count,20000.000000,20000.000000,20000.00000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,...,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000
mean,49.433700,0.499200,0.50150,5.236552,2.495150,0.501050,0.503000,0.504850,70.131699,1.547701,...,0.579300,0.577400,0.573850,0.578650,0.248350,0.249500,0.255900,0.253400,0.248950,0.250800
std,17.389296,0.500012,0.50001,2.754359,1.115992,0.500011,0.500004,0.499989,17.270834,0.835995,...,0.493684,0.493985,0.494528,0.493788,0.432067,0.432734,0.436377,0.434969,0.432416,0.433484
min,20.000000,0.000000,0.00000,0.500410,1.000000,0.000000,0.000000,0.000000,40.002386,0.100017,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,34.000000,0.000000,0.00000,2.836686,2.000000,0.000000,0.000000,0.000000,55.224439,0.830433,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,49.000000,0.000000,1.00000,5.200675,2.000000,1.000000,1.000000,1.000000,70.305066,1.541821,...,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,65.000000,1.000000,1.00000,7.624299,3.000000,1.000000,1.000000,1.000000,85.043395,2.262484,...,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000
max,79.000000,1.000000,1.00000,9.999420,4.000000,1.000000,1.000000,1.000000,99.998125,2.999835,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [13]:
corr_matrix = X.corr().abs()
corr_matrix

,age,gender,tumor_type,tumor_size,stage,radiation_treatment,surgery_performed,chemotherapy,tumor_growth_rate,family_history,...,seizures,vision_issues,nausea,headache,location_occipital,location_parietal,location_temporal,histology_glioblastoma,histology_medulloblastoma,histology_meningioma
age,1.000000,0.004163,0.005734,0.014385,0.001871,0.002548,0.007850,0.005843,0.009853,0.005474,...,0.004251,0.010567,0.002347,0.005764,0.005496,0.001926,0.006772,0.005090,0.001839,0.002690
gender,0.004163,1.000000,0.003795,0.008140,0.000844,0.002303,0.002210,0.006516,0.002592,0.001000,...,0.009264,0.007336,0.001075,0.004609,0.000353,0.007628,0.006667,0.004151,0.009594,0.010148
tumor_type,0.005734,0.003795,1.000000,0.014354,0.009620,0.009494,0.003818,0.004671,0.000502,0.000201,...,0.001292,0.014511,0.006413,0.005237,0.009710,0.000812,0.014053,0.000598,0.003924,0.004724
tumor_size,0.014385,0.008140,0.014354,1.000000,0.009682,0.015159,0.005077,0.010469,0.004347,0.007195,...,0.006064,0.008058,0.001625,0.008125,0.018037,0.001249,0.000282,0.008104,0.014157,0.005459
stage,0.001871,0.000844,0.009620,0.009682,1.000000,0.008880,0.005216,0.004881,0.001872,0.004256,...,0.016682,0.004082,0.000558,0.000329,0.002117,0.012135,0.007924,0.007476,0.003663,0.006685
radiation_treatment,0.002548,0.002303,0.009494,0.015159,0.008880,1.000000,0.003913,0.010380,0.009088,0.006300,...,0.000439,0.002050,0.004965,0.001691,0.002149,0.008606,0.001461,0.008663,0.002018,0.000523
surgery_performed,0.007850,0.002210,0.003818,0.005077,0.005216,0.003913,1.000000,0.002842,0.001890,0.002799,...,0.001572,0.005596,0.007092,0.000158,0.001990,0.007171,0.002602,0.002806,0.000361,0.003010
chemotherapy,0.005843,0.006516,0.004671,0.010469,0.004881,0.010380,0.002842,1.000000,0.004694,0.016103,...,0.004901,0.016397,0.001180,0.014507,0.005460,0.003420,0.005770,0.006111,0.001238,0.002693
tumor_growth_rate,0.009853,0.002592,0.000502,0.004347,0.001872,0.009088,0.001890,0.004694,1.000000,0.001102,...,0.003128,0.003400,0.001301,0.011458,0.007351,0.005232,0.003304,0.000387,0.001107,0.002932
family_history,0.005474,0.001000,0.000201,0.007195,0.004256,0.006300,0.002799,0.016103,0.001102,1.000000,...,0.000981,0.007927,0.004520,0.000474,0.000231,0.000115,0.003779,0.003102,0.012489,0.001961


In [14]:
# Keep only the upper triangle (excluding the diagonal)
upper = np.triu(corr_matrix, k=1)

# Get column names
colnames = corr_matrix.columns

# Find columns to drop
to_drop = [colnames[i] for i in range(len(colnames)) if any(upper[:, i] > 0.9)]

# Drop them
X = X.drop(columns=to_drop)

In [15]:
X.shape

(20000, 22)

In [16]:
X.head()

,age,gender,tumor_type,tumor_size,stage,radiation_treatment,surgery_performed,chemotherapy,tumor_growth_rate,family_history,...,seizures,vision_issues,nausea,headache,location_occipital,location_parietal,location_temporal,histology_glioblastoma,histology_medulloblastoma,histology_meningioma
0,73,1,1,5.375612,3,0,0,0,0.111876,0,...,1,1,0,0,0,0,1,0,0,0
1,26,1,0,4.847098,2,1,1,1,2.165736,1,...,0,0,1,1,0,1,0,1,0,0
2,31,1,0,5.588391,1,0,0,0,1.884228,0,...,1,1,0,1,0,1,0,0,0,1
3,29,1,1,1.436600,4,1,0,1,1.283342,1,...,1,1,0,1,0,0,1,0,1,0
4,54,0,0,2.417506,1,0,0,1,2.069477,0,...,1,0,0,1,0,1,0,1,0,0


# regression

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y_num)

## vanilla

In [18]:
regression_pipelines = {
    'linreg': make_pipeline(StandardScaler(), LinearRegression()),
    'rf': make_pipeline(RandomForestRegressor()),
    'xgb': make_pipeline(XGBRegressor())
}

In [19]:
for key, val in regression_pipelines.items():
    val.fit(X_train, y_train)
    y_pred = val.predict(X_test)
    rmse = mean_squared_error(y_test, y_pred)
    print(f'RMSE for {key}: {rmse}')

RMSE for linreg: 299.1626168777181
RMSE for rf: 306.78823290098325
RMSE for xgb: 322.16347033100465


In [20]:
rmse_scorer = make_scorer(mean_squared_error, greater_is_better=False)

xgbr_params = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5],
    'learning_rate': [.05, .01]
}

grid_xgbr = GridSearchCV(XGBRegressor(), param_grid = xgbr_params, scoring=rmse_scorer, cv=5)
grid_xgbr.fit(X_train, y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    gamma=None, gpu_id=None, grow_policy=None,
                                    importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, max_...
                                    n_jobs=None, num_parallel_tree=None,
                                    objective='reg:squarederror',
                                    predictor=None, random_state=None,
                                    reg_alpha=None, ...),
             iid='deprec

In [21]:
print(f'best rmse: {-grid_xgbr.best_score_}')
grid_xgbr.best_estimator_

best rmse: 299.3096437736617


XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.05, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=3, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, objective='reg:squarederror',
             predictor='auto', random_state=0, reg_alpha=0, ...)

## deep learning

In [22]:
model = Sequential()

model.add(Dense(64, input_dim = X_train.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1))

model.compile(optimizer=Adam(learning_rate=.001), loss='mse')

early_stopping = EarlyStopping(monitor='val_loss', patience=10)

model.fit(X_train, y_train, epochs=100, batch_size=32, callbacks=[early_stopping], validation_split=.2)

Epoch 1/100
375/375 [==============================] - 1s 2ms/step - loss: 842.0393 - val_loss: 473.4038
Epoch 2/100
375/375 [==============================] - 0s 1ms/step - loss: 414.6501 - val_loss: 389.8310
Epoch 3/100
375/375 [==============================] - 0s 1ms/step - loss: 382.0484 - val_loss: 376.9750
Epoch 4/100
375/375 [==============================] - 0s 1ms/step - loss: 372.1151 - val_loss: 368.6605
Epoch 5/100
375/375 [==============================] - 0s 1ms/step - loss: 364.8851 - val_loss: 360.7932
Epoch 6/100
375/375 [==============================] - 0s 1ms/step - loss: 357.6754 - val_loss: 355.6950
Epoch 7/100
375/375 [==============================] - 0s 1ms/step - loss: 351.7607 - val_loss: 351.2576
Epoch 8/100
375/375 [==============================] - 0s 1ms/step - loss: 347.2824 - val_loss: 350.1834
Epoch 9/100
375/375 [==============================] - 0s 1ms/step - loss: 341.8217 - val_loss: 343.8494
Epoch 10/100
375/375 [==============================] -

In [23]:
y_pred = model.predict(X_test)
y_pred

157/157 [==============================] - 0s 902us/step


array([[69.110085],
       [70.02508 ],
       [72.16898 ],
       ...,
       [71.19998 ],
       [71.264465],
       [68.89457 ]], dtype=float32)

# classification

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y_cat)

## vanilla

In [25]:
classification_pipelines = {
    'logreg': make_pipeline(StandardScaler(), LogisticRegression(multi_class='multinomial')),
    'xgb': make_pipeline(XGBClassifier(objective='multi:softprob', num_class = 3))
}

In [26]:
for key, val in classification_pipelines.items():
    val.fit(X_train, y_train)
    y_pred = val.predict(X_test)
    f1 = f1_score(y_test, y_pred, average='weighted')
    print(f'f1_score for {key}: {f1}')

f1_score for logreg: 0.39318463680019905
f1_score for xgb: 0.38443216594406004


In [27]:
f1_scorer = make_scorer(f1_score, average='macro')

xgbc_params = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5],
    'learning_rate': [.05, .01]
}

grid_xgbc = GridSearchCV(XGBClassifier(), param_grid = xgbc_params, scoring=f1_scorer, cv=5)
grid_xgbc.fit(X_train, y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     gamma=None, gpu_id=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None, max...
                                     n_estimators=100, n_jobs=None,
                                     num_parallel_tree=None,
                                     objective='binary:logistic',
                                     predictor=None, random_state=None,
                

## deep learning

In [28]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [29]:
model = Sequential()

model.add(Dense(64, input_dim = X_train.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(3, activation='softmax'))

model.compile(optimizer=Adam(learning_rate=.001), loss='categorical_crossentropy')

early_stopping = EarlyStopping(monitor='val_loss', patience=10)

model.fit(X_train, y_train, epochs=100, batch_size=32, callbacks=[early_stopping], validation_split=.2)

Epoch 1/100
375/375 [==============================] - 1s 2ms/step - loss: 1.0767 - val_loss: 1.0379
Epoch 2/100
375/375 [==============================] - 1s 2ms/step - loss: 1.0472 - val_loss: 1.0361
Epoch 3/100
375/375 [==============================] - 1s 2ms/step - loss: 1.0481 - val_loss: 1.0588
Epoch 4/100
375/375 [==============================] - 1s 2ms/step - loss: 1.0442 - val_loss: 1.0318
Epoch 5/100
375/375 [==============================] - 1s 2ms/step - loss: 1.0413 - val_loss: 1.0268
Epoch 6/100
375/375 [==============================] - 1s 1ms/step - loss: 1.0413 - val_loss: 1.0323
Epoch 7/100
375/375 [==============================] - 1s 1ms/step - loss: 1.0409 - val_loss: 1.0660
Epoch 8/100
375/375 [==============================] - 1s 1ms/step - loss: 1.0386 - val_loss: 1.0295
Epoch 9/100
375/375 [==============================] - 1s 1ms/step - loss: 1.0387 - val_loss: 1.0329
Epoch 10/100
375/375 [==============================] - 1s 1ms/step - loss: 1.0371 - val_lo

In [30]:
model.predict(X_test)

157/157 [==============================] - 0s 819us/step


array([[0.18245241, 0.37573808, 0.4418095 ],
       [0.1693702 , 0.3261505 , 0.50447935],
       [0.19370772, 0.32002452, 0.48626772],
       ...,
       [0.17880586, 0.31831276, 0.5028814 ],
       [0.17634225, 0.3072327 , 0.51642513],
       [0.17345837, 0.316586  , 0.5099556 ]], dtype=float32)